In [ ]:
# Exercicio OK

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
import pyspark.sql.functions as f

In [2]:
spark = SparkSession.builder \
    .appName("improve-enrich-data") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

In [3]:
## versin pyspark

print("Versão do PySpark:", spark.version)

Versão do PySpark: 3.4.0


In [4]:
# Definir o schema dos dados na leitura
schema = StructType([
    StructField("invoice_no", StringType(), True),
    StructField("invoice_date", StringType(), True), # observe que é um tipo string para uma coluna de datetime ;)
    StructField("customer_id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("category", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("selling_price_per_unit", DoubleType(), True),
    StructField("cost_price_per_unit", DoubleType(), True),
    StructField("payment_method", StringType(), True),
    StructField("region", StringType(), True),
    StructField("state", StringType(), True),
    StructField("shopping_mall", StringType(), True)
])

In [5]:
'''
Sobre o Conjunto de Dados
Visão Geral

Este conjunto de dados é uma criação fictícia, projetada especificamente para ajudar analistas de 
dados a praticar análise exploratória de dados e visualização de dados. 
Ele contém informações sobre pedidos feitos por clientes em um aplicativo de entrega de produtos de 
supermercado.
'''
df = spark.read.csv("datasource/Different_stores_dataset.csv", header=True, schema=schema)

In [6]:
df.show(n=1, truncate=False, vertical=True)

-RECORD 0---------------------------------
 invoice_no             | I138884         
 invoice_date           | 10/30/2016 9:58 
 customer_id            | C241288         
 gender                 | Female          
 age                    | 28              
 category               | Clothing        
 quantity               | 5               
 selling_price_per_unit | 1500.4          
 cost_price_per_unit    | 1425.38         
 payment_method         | Credit Card     
 region                 | South           
 state                  | Kentucky        
 shopping_mall          | Kanyon          
only showing top 1 row



In [7]:
# from ydata_profiling import ProfileReport
# '''
# Esta linha importa a classe ProfileReport do módulo ydata_profiling que é uma biblioteca para análise exploratória de dados. 
# Seu objetivo é gerar relatórios de perfil de conjuntos de dados, oferecendo uma visão geral abrangente dos dados.
# '''
# report = ProfileReport(
#     df,
#     title='Different stores dataset',       
#     dataset={
#         "description": "Dados fictícios sobre pedidos feitos por clientes em um aplicativo de entrega de produtos de supermercado",
#         "copyright_holder": "Kushal Kamra",
#         "url": "https://www.kaggle.com/datasets/kzmontage/sales-from-different-stores",
#     },
#     variables={
#         "descriptions": {
#             "invoice_no": "Número da fatura associado a cada transação.",
#             "customer_id": "Identificador único para cada cliente.",
#             "gender": "Gênero do cliente (presumido ser binário: masculino/feminino).",
#             "age": "Idade do cliente.",
#             "category": "Categoria do produto associada à transação.",
#             "quantity": "Quantidade de produtos comprados em cada transação.",
#             "selling_price_per_unit": "Preço de venda por unidade do produto.",
#             "cost_price_per_unit": "Preço de custo por unidade do produto.",
#             "payment_method": "Método de pagamento usado na transação.",
#             "region": "Região geográfica associada à transação.",
#             "state": "Estado onde a transação ocorreu.",
#             "shopping_mall": "Shopping center onde ocorreu a transação.",
#         }
#     },
#     infer_dtypes=False,
#     interactions=None,
#     missing_diagrams=None,
#     correlations={"pearson": {"calculate": True}}
# )

In [8]:
# report

In [9]:
# Limpeza e padronização dos dados: Vamos melhorar o schema dos dados modificando o _Tipo_ da coluna `invoice_date` de String para Timestamp. 
df = df.withColumn("invoice_datetime", f.to_timestamp(df.invoice_date, 'MM/dd/yyyy H:mm'))

In [10]:
# Padronização de Dados: Criar uma coluna extra invoice_date do _Tipo_ String no formato 'dd/MM/yyyy' para contrução de partição 
# melhorando o desempenho de consultas com data.
df = df.withColumn("invoice_date", f.date_format("invoice_datetime", 'dd/MM/yyyy'))

In [11]:
df.select("invoice_datetime", "invoice_date").show()

+-------------------+------------+
|   invoice_datetime|invoice_date|
+-------------------+------------+
|2016-10-30 09:58:00|  30/10/2016|
|2016-10-30 09:58:00|  30/10/2016|
|2016-10-30 09:58:00|  30/10/2016|
|2016-10-30 09:58:00|  30/10/2016|
|2016-10-30 09:59:00|  30/10/2016|
|2016-10-30 09:59:00|  30/10/2016|
|2016-10-30 09:59:00|  30/10/2016|
|2016-10-30 09:59:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:01:00|  30/10/2016|
|2016-10-30 10:01:00|  30/10/2016|
|2016-10-30 10:01:00|  30/10/2016|
|2016-10-30 10:01:00|  30/10/2016|
|2016-10-30 10:02:00|  30/10/2016|
|2016-10-30 10:02:00|  30/10/2016|
|2016-10-30 10:02:00|  30/10/2016|
+-------------------+------------+
only showing top 20 rows



In [12]:
df.createOrReplaceTempView("stores")

In [13]:
spark.sql(
    "SELECT * FROM stores"
).show(n=1, vertical=True)

-RECORD 0-------------------------------------
 invoice_no             | I138884             
 invoice_date           | 30/10/2016          
 customer_id            | C241288             
 gender                 | Female              
 age                    | 28                  
 category               | Clothing            
 quantity               | 5                   
 selling_price_per_unit | 1500.4              
 cost_price_per_unit    | 1425.38             
 payment_method         | Credit Card         
 region                 | South               
 state                  | Kentucky            
 shopping_mall          | Kanyon              
 invoice_datetime       | 2016-10-30 09:58:00 
only showing top 1 row



In [14]:
'''
1. Melhoramento e Enriquecimento de Dados:
    Você poderia enriquecer esses dados adicionando informações de rentabilidade para cada 
    transação (calculando a diferença entre selling_price_per_unit e cost_price_per_unit
    multiplicada pela quantity).
'''

'\n1. Melhoramento e Enriquecimento de Dados:\n    Você poderia enriquecer esses dados adicionando informações de rentabilidade para cada \n    transação (calculando a diferença entre selling_price_per_unit e cost_price_per_unit\n    multiplicada pela quantity).\n'

In [15]:
df.columns

['invoice_no',
 'invoice_date',
 'customer_id',
 'gender',
 'age',
 'category',
 'quantity',
 'selling_price_per_unit',
 'cost_price_per_unit',
 'payment_method',
 'region',
 'state',
 'shopping_mall',
 'invoice_datetime']

In [16]:
df.limit(3).show()

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+--------------+------+----------+--------------+-------------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit|payment_method|region|     state| shopping_mall|   invoice_datetime|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+--------------+------+----------+--------------+-------------------+
|   I138884|  30/10/2016|    C241288|Female| 28|Clothing|       5|                1500.4|            1425.38|   Credit Card| South|  Kentucky|        Kanyon|2016-10-30 09:58:00|
|   I317333|  30/10/2016|    C111565|  Male| 21|   Shoes|       3|               1800.51|           1620.459|    Debit Card| South|  Kentucky|Viaport Outlet|2016-10-30 09:58:00|
|   I127801|  30/10/2016|    C266599|  Male| 20|Clothing|       1|                300.08|             295.06| 

In [17]:
from pyspark.sql.functions import col

In [18]:
df = df.withColumn("transacao", (col("selling_price_per_unit") - col("cost_price_per_unit")) * col("quantity"))

In [19]:
df.limit(2).show()

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+--------------+------+--------+--------------+-------------------+-----------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit|payment_method|region|   state| shopping_mall|   invoice_datetime|        transacao|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+--------------+------+--------+--------------+-------------------+-----------------+
|   I138884|  30/10/2016|    C241288|Female| 28|Clothing|       5|                1500.4|            1425.38|   Credit Card| South|Kentucky|        Kanyon|2016-10-30 09:58:00|375.0999999999999|
|   I317333|  30/10/2016|    C111565|  Male| 21|   Shoes|       3|               1800.51|           1620.459|    Debit Card| South|Kentucky|Viaport Outlet|2016-10-30 09:58:00|540.1529999999998|
+----------+------------+-----

In [20]:
''' 
2. Eliminação de Dados Irrelevantes:
    Eliminar dados irrelevantes significa remover informações que não contribuem para a análise ou 
    objetivo específico.

    Se o objetivo for analisar apenas as vendas de produtos, campos como invoice_no e customer_id 
    podem ser considerados irrelevantes e, portanto, removidos.
    Em análises focadas em tendências, detalhes como shopping_mall podem ser descartados.
'''

' \n2. Eliminação de Dados Irrelevantes:\n    Eliminar dados irrelevantes significa remover informações que não contribuem para a análise ou \n    objetivo específico.\n\n    Se o objetivo for analisar apenas as vendas de produtos, campos como invoice_no e customer_id \n    podem ser considerados irrelevantes e, portanto, removidos.\n    Em análises focadas em tendências, detalhes como shopping_mall podem ser descartados.\n'

In [21]:
####  remove alguma coluna 
df.show(3)

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+--------------+------+----------+--------------+-------------------+-----------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit|payment_method|region|     state| shopping_mall|   invoice_datetime|        transacao|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+--------------+------+----------+--------------+-------------------+-----------------+
|   I138884|  30/10/2016|    C241288|Female| 28|Clothing|       5|                1500.4|            1425.38|   Credit Card| South|  Kentucky|        Kanyon|2016-10-30 09:58:00|375.0999999999999|
|   I317333|  30/10/2016|    C111565|  Male| 21|   Shoes|       3|               1800.51|           1620.459|    Debit Card| South|  Kentucky|Viaport Outlet|2016-10-30 09:58:00|540.1529999999998|
|   I127801|  30/10/

In [22]:
df = df.drop("payment_method")

In [23]:
df.limit(1).show()

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+-------------+-------------------+-----------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit|region|   state|shopping_mall|   invoice_datetime|        transacao|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+-------------+-------------------+-----------------+
|   I138884|  30/10/2016|    C241288|Female| 28|Clothing|       5|                1500.4|            1425.38| South|Kentucky|       Kanyon|2016-10-30 09:58:00|375.0999999999999|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+-------------+-------------------+-----------------+



In [24]:
'''
3. Granulação e Agregação:
    A granulação e agregação envolvem resumir ou combinar dados para um nível de detalhe mais 
    alto ou mais baixo, dependendo da necessidade da análise.

    3.1 Granulação: Filtrar dados para um nível mais detalhado, como todas as transações de um 
    cliente específico (customer_id).

    3.2 Agregação: Sumarizar dados, como calcular o total de vendas por category ou a média de 
    idade dos clientes (age) por gender.
'''

'\n3. Granulação e Agregação:\n    A granulação e agregação envolvem resumir ou combinar dados para um nível de detalhe mais \n    alto ou mais baixo, dependendo da necessidade da análise.\n\n    3.1 Granulação: Filtrar dados para um nível mais detalhado, como todas as transações de um \n    cliente específico (customer_id).\n\n    3.2 Agregação: Sumarizar dados, como calcular o total de vendas por category ou a média de \n    idade dos clientes (age) por gender.\n'

In [25]:
from pyspark.sql.functions import sum , round

In [26]:
#3.1 Granulação: Filtrar dados para um nível mais detalhado, como todas as transações de um 
#    cliente específico (customer_id).

#df.show()

#df_agrupado = df.groupBy("customer_id").agg(sum("transacao").alias("total_valor_compra"))
df_agrupado = df.groupBy("customer_id").agg(round(sum("transacao"),2).alias("total_transacao"))

In [27]:
df_agrupado.show()

+-----------+---------------+
|customer_id|total_transacao|
+-----------+---------------+
|    C236859|         2625.0|
|    C513561|          14.12|
|    C294024|          375.1|
|    C248884|          48.79|
|    C113739|         240.06|
|    C216369|           1.57|
|    C182606|        1500.43|
|    C323469|          203.3|
|    C299567|          14.12|
|    C128277|          203.3|
|    C761041|          43.01|
|    C239313|          203.3|
|    C128876|         150.04|
|    C475080|         300.08|
|    C632239|         300.08|
|    C280151|         675.18|
|    C274780|           60.6|
|    C967744|         540.15|
|    C286502|          105.0|
|    C319808|         150.04|
+-----------+---------------+
only showing top 20 rows



In [28]:
#3.2 Agregação: Sumarizar dados, como calcular o total de vendas por category ou a média de 
#    idade dos clientes (age) por gender.


resultado = df.groupBy("gender").agg({"age": "avg"})

In [29]:
resultado.show()

+------+------------------+
|gender|          avg(age)|
+------+------------------+
|Female| 43.45351534918127|
|  Male|43.387767354596626|
+------+------------------+



In [30]:
'''
4. Consistência, Integridade, Concordância:
    Esses conceitos referem-se a garantir que os dados sejam precisos, confiáveis e 
    consistentes em todo o conjunto de dados.

    Verificar se todos os invoice_date estão no formato correto e se os valores de age 
    são razoáveis (por exemplo, não negativos ou excessivamente altos).
    Assegurar que selling_price_per_unit seja sempre maior que cost_price_per_unit, 
    pois o contrário indicaria uma inconsistência nos dados
'''

'\n4. Consistência, Integridade, Concordância:\n    Esses conceitos referem-se a garantir que os dados sejam precisos, confiáveis e \n    consistentes em todo o conjunto de dados.\n\n    Verificar se todos os invoice_date estão no formato correto e se os valores de age \n    são razoáveis (por exemplo, não negativos ou excessivamente altos).\n    Assegurar que selling_price_per_unit seja sempre maior que cost_price_per_unit, \n    pois o contrário indicaria uma inconsistência nos dados\n'

In [31]:
data_formato_correto = r'^\d{2}/\d{2}/\d{4}$'

In [32]:
from pyspark.sql.functions import col, when

In [33]:
df_resultado = df.withColumn(
    "formato_correto",
    when(col("invoice_date").rlike(data_formato_correto), "Correto").otherwise("Incorreto")
)

In [34]:
df_resultado.show(2)

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+--------------+-------------------+-----------------+---------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit|region|   state| shopping_mall|   invoice_datetime|        transacao|formato_correto|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+--------------+-------------------+-----------------+---------------+
|   I138884|  30/10/2016|    C241288|Female| 28|Clothing|       5|                1500.4|            1425.38| South|Kentucky|        Kanyon|2016-10-30 09:58:00|375.0999999999999|        Correto|
|   I317333|  30/10/2016|    C111565|  Male| 21|   Shoes|       3|               1800.51|           1620.459| South|Kentucky|Viaport Outlet|2016-10-30 09:58:00|540.1529999999998|        Correto|
+----------+------------+

In [35]:
# Definir limites razoáveis para a idade
idade_minima = 0
idade_maxima = 100

In [36]:
df_resultado = df.withColumn(
    "idade_razoavel",
    when((col("age") >= idade_minima) & (col("age") <= idade_maxima), "Razoavel").otherwise("Nao Razoavel")
)

In [37]:
df_resultado.show(2)

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+--------------+-------------------+-----------------+--------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit|region|   state| shopping_mall|   invoice_datetime|        transacao|idade_razoavel|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+--------------+-------------------+-----------------+--------------+
|   I138884|  30/10/2016|    C241288|Female| 28|Clothing|       5|                1500.4|            1425.38| South|Kentucky|        Kanyon|2016-10-30 09:58:00|375.0999999999999|      Razoavel|
|   I317333|  30/10/2016|    C111565|  Male| 21|   Shoes|       3|               1800.51|           1620.459| South|Kentucky|Viaport Outlet|2016-10-30 09:58:00|540.1529999999998|      Razoavel|
+----------+------------+-----

In [38]:
df_resultado = df.withColumn(
    "preco_consistente",
    when(col("selling_price_per_unit") > col("cost_price_per_unit"), "Consistente").otherwise("Inconsistente")
)

In [39]:
df_inconsistente = df_resultado.filter(col("preco_consistente") == "Inconsistente")

In [40]:
df_resultado.show(2)

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+--------------+-------------------+-----------------+-----------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit|region|   state| shopping_mall|   invoice_datetime|        transacao|preco_consistente|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+------+--------+--------------+-------------------+-----------------+-----------------+
|   I138884|  30/10/2016|    C241288|Female| 28|Clothing|       5|                1500.4|            1425.38| South|Kentucky|        Kanyon|2016-10-30 09:58:00|375.0999999999999|      Consistente|
|   I317333|  30/10/2016|    C111565|  Male| 21|   Shoes|       3|               1800.51|           1620.459| South|Kentucky|Viaport Outlet|2016-10-30 09:58:00|540.1529999999998|      Consistente|
+----------+---

In [41]:
df_inconsistente.show(2)

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+-------+--------------+-------------+-------------------+------------------+-----------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit| region|         state|shopping_mall|   invoice_datetime|         transacao|preco_consistente|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+-------+--------------+-------------+-------------------+------------------+-----------------+
|   I640508|  30/10/2016|    C225330|Female| 67|    Toys|       4|                143.36|            144.688|  South|North Carolina|    Metrocity|2016-10-30 10:00:00|-5.311999999999898|    Inconsistente|
|   I985478|  30/10/2016|    C324683|  Male| 55|Clothing|       4|               1200.32|           1240.304|Central|          Iowa|       Kanyon|2016-10-30 10:21:00|-159.9360000000006

In [42]:
'''
5. Duplicação e Redundância:
    Dados duplicados ou redundantes podem distorcer análises e devem ser removidos ou tratados.
    
    Procurar por registros duplicados (por exemplo, duas transações com o mesmo invoice_no).
    Identificar redundâncias, como se a combinação de region e state sempre leva ao 
    mesmo shopping_mall, indicando que um desses campos pode ser redundante.
'''

'\n5. Duplicação e Redundância:\n    Dados duplicados ou redundantes podem distorcer análises e devem ser removidos ou tratados.\n    \n    Procurar por registros duplicados (por exemplo, duas transações com o mesmo invoice_no).\n    Identificar redundâncias, como se a combinação de region e state sempre leva ao \n    mesmo shopping_mall, indicando que um desses campos pode ser redundante.\n'

In [43]:
df_sem_duplicatas = df.dropDuplicates(["invoice_no"])

In [44]:
df_sem_duplicatas.show(2)

+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+-------+-------+-------------+-------------------+-----------------+
|invoice_no|invoice_date|customer_id|gender|age|category|quantity|selling_price_per_unit|cost_price_per_unit| region|  state|shopping_mall|   invoice_datetime|        transacao|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+-------+-------+-------------+-------------------+-----------------+
|   I100008|  15/11/2016|    C199951|  Male| 65|Clothing|       5|                1500.4|            1425.38|  South|Alabama| Zorlu Center|2016-11-15 09:44:00|375.0999999999999|
|   I100024|  14/11/2016|    C244411|Female| 67|   Books|       3|                 45.45|             31.815|Central|  Texas| Zorlu Center|2016-11-14 18:13:00|           40.905|
+----------+------------+-----------+------+---+--------+--------+----------------------+-------------------+-

In [45]:
# Identificar redundâncias na combinação de region e state
redundancias = df.groupBy("region", "state", "shopping_mall").count().filter(col("count") > 1)

In [46]:
redundancias.show(5)

+------+----------+-------------+-----+
|region|     state|shopping_mall|count|
+------+----------+-------------+-----+
| South|   Florida|    Metrocity|  553|
|  East|  Delaware| Istinye Park|  562|
|  West|California|       Kanyon| 3849|
| South| Tennessee| Zorlu Center|  199|
|  West|    Oregon|       Kanyon|  342|
+------+----------+-------------+-----+
only showing top 5 rows

